# Многопоточность в Python

## 1. Single process

In [1]:
import time


def fib(n: int) -> int:
    return fib(n-1) + fib(n-2) if n > 2 else 1


final_fibonacci_number = 40

tasks = list(range(1, final_fibonacci_number + 1))
start_time = time.perf_counter()
answers = []
for number in tasks:
    answers.append(fib(number))
finish_time = time.perf_counter()
print(*answers)
print('Duration:', finish_time - start_time)

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155
Duration: 57.268999982625246


### Распределение задач между потоками при помощи очереди
Queue предоставляет нам механизм взаимодействия потоков между процессами FIFO (первым пришел — первым обслужен).
Ниже пример, где "воркеры" выполняют вычисление чисел Фибоначчи.
## 2. With `multiprocessing.Queue` for tasks
Воркеры "добывают" себе задание из очереди и печатают результат на экран.

In [2]:
from multiprocessing import Process, Queue
import os
import time


def worker(tasks: Queue, process_index: int):
    def fib(n: int) -> int:
        return fib(n-1) + fib(n-2) if n > 2 else 1
    
    while not tasks.empty():  # пока очередь не пуста выполняем одну очередную задачу
        number = tasks.get()
        answer = fib(number)
        print(f"worker {process_index}, PID={os.getpid()}: fib({number}) = {answer}")


workers_number = 4
final_fibonacci_number = 40

tasks = Queue()
for n in range(1, final_fibonacci_number + 1):
    tasks.put(n)

workers = []
for process_index in range(workers_number):
    worker_process = Process(target=worker, args=(tasks, process_index,))
    workers.append(worker_process)
start_time = time.perf_counter()
for worker_process in workers:
    worker_process.start()
for worker_process in workers:
    worker_process.join()
finish_time = time.perf_counter()
print('Duration:', finish_time - start_time)

worker 0, PID=246855: fib(1) = 1
worker 0, PID=246855: fib(2) = 1
worker 1, PID=246856: fib(3) = 2
worker 0, PID=246855: fib(4) = 3
worker 1, PID=246856: fib(5) = 5
worker 0, PID=246855: fib(6) = 8
worker 2, PID=246861: fib(7) = 13
worker 1, PID=246856: fib(8) = 21
worker 0, PID=246855: fib(9) = 34
worker 1, PID=246856: fib(11) = 89
worker 2, PID=246861: fib(10) = 55
worker 0, PID=246855: fib(13) = 233
worker 1, PID=246856: fib(14) = 377
worker 3, PID=246870: fib(12) = 144
worker 0, PID=246855: fib(16) = 987
worker 1, PID=246856: fib(17) = 1597
worker 3, PID=246870: fib(18) = 2584
worker 0, PID=246855: fib(19) = 4181
worker 1, PID=246856: fib(20) = 6765
worker 2, PID=246861: fib(15) = 610
worker 3, PID=246870: fib(21) = 10946
worker 1, PID=246856: fib(22) = 17711
worker 0, PID=246855: fib(23) = 28657
worker 2, PID=246861: fib(24) = 46368
worker 3, PID=246870: fib(25) = 75025
worker 1, PID=246856: fib(26) = 121393
worker 0, PID=246855: fib(27) = 196418
worker 2, PID=246861: fib(28) = 31

## 3. With `multiprocessing.Queue` for tasks and answers
Теперь не будем печатать результаты на экран, но сделаем очередь результатов:

In [3]:
from multiprocessing import Process, Queue
import os
import time


def worker(tasks: Queue, answers: Queue, process_index: int):
    def fib(n: int) -> int:
        return fib(n-1) + fib(n-2) if n > 2 else 1
    
    while not tasks.empty():  # пока очередь не пуста выполняем одну очередную задачу
        number = tasks.get()
        answer = fib(number)
        answers.put((process_index, os.getpid(), number, answer))

        
workers_number = 4
final_fibonacci_number = 40

tasks = Queue()
answers = Queue()
for n in range(1, final_fibonacci_number + 1):
    tasks.put(n)

workers = []
for process_index in range(workers_number):
    worker_process = Process(target=worker,
                             args = (tasks, answers, process_index))
    workers.append(worker_process)

start_time = time.perf_counter()
for worker_process in workers:
    worker_process.start()
for worker_process in workers:
    worker_process.join()   
finish_time = time.perf_counter()

ordered_answers = []
while not answers.empty():
    process_index, pid, number, answer = answers.get()
    ordered_answers.append((number, answer))
ordered_answers.sort()
print(*(answer for number, answer in ordered_answers))
print('Duration:', finish_time - start_time)

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155
Duration: 28.084279399365187


## 4. With `multiprocessing.Pool`
Откажемся от явного использования `Process` и `Queue` в пользу `Pool`.

In [4]:
from multiprocessing import Pool
import os
import time


def fib(n: int) -> int:
    return fib(n-1) + fib(n-2) if n > 2 else 1
    

workers_number = 4
final_fibonacci_number = 40

tasks = list(range(1, final_fibonacci_number + 1))
start_time = time.perf_counter()
with Pool(workers_number) as pool_of_processes:
    answers = list(pool_of_processes.map(fib, tasks))
finish_time = time.perf_counter()
print(*answers)
print('Duration:', finish_time - start_time)

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155
Duration: 27.256503265351057


## 5. C++ version

In [5]:
!g++ fibonacci.cpp

In [6]:
!time ./a.out

1 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 
1.05user 0.00system 0:01.06elapsed 99%CPU (0avgtext+0avgdata 3324maxresident)k
0inputs+0outputs (0major+122minor)pagefaults 0swaps


## 6. Python plus C++

In [7]:
!g++ -c -shared -fPIC fibonacci_func.cpp

In [8]:
!make

g++ -shared fibonacci_func.o -o fibonacci_func.so -Wl,--whole-archive -Wl,--no-whole-archive


In [9]:
import ctypes
from multiprocessing import Pool
import os
import time


def fib(n: int) -> int:
    return int(fibc.fib(ctypes.c_int64(n)))   # вызываю функцию на языке С++


workers_number = 4
final_fibonacci_number = 40

fibc = ctypes.CDLL(os.path.abspath('fibonacci_func.so'))
tasks = list(range(1, final_fibonacci_number))
start_time = time.perf_counter()
with Pool(workers_number) as pool_of_processes:
        answers = list(pool_of_processes.map(fib, tasks))
finish_time = time.perf_counter()
print(*answers)
print('Duration:', finish_time - start_time)

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986
Duration: 0.8425645679235458


## 7. KISS

In [10]:
import time

final_fibonacci_number = 40

start_time = time.perf_counter()
fib = [0, 1] + [None]*(final_fibonacci_number - 1)
for i in range(2, final_fibonacci_number + 1):
    fib[i] = fib[i-1] + fib[i-2]
for i in range(0, final_fibonacci_number + 1):
    print(fib[i], end=' ')
finish_time = time.perf_counter()
print('\nDuration:', finish_time - start_time)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155 
Duration: 0.0009240768849849701


## 8. With `numba`

In [11]:
!pip3 install --user --quiet numba

You are using pip version 18.1, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [12]:
import time

from numba import jit


@jit(nopython=True)
def fib(n: int) -> int:
    return fib(n-1) + fib(n-2) if n > 2 else 1


final_fibonacci_number = 40

tasks = list(range(1, final_fibonacci_number + 1))
start_time = time.perf_counter()
answers = []
for number in tasks:
    answers.append(fib(number))
finish_time = time.perf_counter()
print(*answers)
print('Duration:', finish_time - start_time)

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155
Duration: 1.6005328372120857
